#  Island Warriors Optimizations


## Problem Description

### Context

In this game, the player embarks on a journey across a series of islands with the objective of minimizing the total adjusted time spent. Each island presents a unique difficulty level, and the time required to overcome an island depends on both its difficulty and the player's power upon arrival. The player's power can be enhanced through:

1. **Character Selection**: Choosing a character with specific starting attributes, including money and modifiers for certain weapon types.
2. **Item Collection**: Picking up items on islands that increase power.
3. **Spending Money**: Investing money on islands to gain additional power.

The player's journey involves strategic decisions regarding:

- **Travel Paths**: Determining the optimal route between islands while considering travel costs.
- **Item Collection**: Deciding which items to collect, given constraints on carrying capacity and item availability.
- **Resource Management**: Balancing spending and saving money to maximize power without exceeding available funds.
---
### Sequence of Actions When Visiting an Island

1. **Arrival at Island**:
   - The player reaches the island as part of their travel path.

2. **Item Collection**:
   - The player decides whether to collect an available item on the island.
   - If an item is collected, it enhances their power and is added to the backpack.

3. **Spending Money**:
   - The player chooses whether to spend money to gain additional power.
   - Spending is limited by available money and must comply with constraints.

4. **Update Player's Power**:
   - The player's power is recalculated based on items collected and money spent.

5. **Adjusted Difficulty Calculation**:
   - The adjusted difficulty of the island is computed based on the player's power.

6. **Overcoming the Island**:
   - The player faces the island's challenges, and the adjusted difficulty affects the time taken.

7. **Preparation for Next Move**:
   - The player plans the next move, considering remaining money, items, and travel costs.

---
### Objective Function
Minimize the total adjusted time:
$$
\text{minimize:} \ \sum_{i} y_i \cdot \frac{\text{Island\_difficulty}_i}{p_i}
$$
Where:
- $y_i$: Binary; 1 if island i is visited, 0 otherwise.
- $p_i$: Total power at island i, calculated as:
  $$
  p_i = \sum_k z_{ik} \cdot \text{item\_value}_{ik} + a \cdot \text{spend\_amount}_i
  $$
Where:
- $a$: Exchange rate for Money and Power
---
### Decision Variables (DVs)

| **Variable**                | **Type**       | **Description**                                                     |
|:---------------------------:|:--------------:|:--------------------------------------------------------------------|
| $x_{ij}$                    | Binary         | $1$ if traveling from island $i$ to $j$, $0$ otherwise.             |
| $y_i$                       | Binary         | $1$ if island $i$ is visited, $0$ otherwise.                        |
| $u_i$                       | Continuous     | Position of island i in the visiting order.                         |
| $z_{ik}$                    | Binary         | $1$ if item $k$ is collected from island $i$, $0$ otherwise.        |
| $b_{ik}$                    | Binary         | $1$ if item $k$ is in the backpack, $0$ otherwise.                  |
| $c_k$                       | Binary         | $1$ if character $k$ is selected, $0$ otherwise.                    |
| $s_i$                       | Binary         | $1$ if money is spent on island $i$, $0$ otherwise.                 |
| $Spend\_Amount_i$           | Continuous     | Amount of money spent on island $i$.                                |
| $p_i$                       | Continuous     | Total power on island $i$.                                          |
| $Adjusted\_Difficulties_i$  | Continuous     | Adjusted difficulty for island $i$.                                 |
| $Money$                     | Continuous     | Total money available.                                              |


---

### Constraints

#### 1. Character Selection
Ensure that exactly one character is selected:
$$
\sum_k c_k = 1
$$

#### 2 Travel Flow
- The number of departures from an island equals the number of arrivals:
  $$
  \sum_j x_{ij} = \sum_j x_{ji} \quad \forall i
  $$
- Start from the designated start island:
  $$
  \sum_j x_{0j} = 1
  $$
- End at the designated end island:
  $$
  \sum_i x_{i, 14} = 1
  $$

- Starting island can only be got from the final island 

$$
\sum_{i \neq 14} x_{i, 0} = 0
$$

$$
x_{14, 0} = 1
$$

- To prevent the player from traveling backward after moving from one island to another through the same route:

$$
x_{ij} + x_{ji} \leq 1 \quad \forall i, j, \; i \neq j
$$

- Player can only reach each island once
$$
\sum_{j} x_{j, i} = y_i, \quad \forall i
$$
$$
\sum_{j} x_{i, j} = y_i, \quad \forall i
$$

#### 3. No Sub Tour: MTZ Constraints
$$
u_i - u_j + n \cdot x_{ij} \leq n - 1 
\quad \forall i,j \in \{1,\ldots,14\}, i \neq j
$$


#### 4. Island Visitation
- Items can only be collected if the island is visited:
  $$
  z_{ik} \leq y_i \quad \forall i, k
  $$
- A player can pick at most one item on each island:
  $$
  \sum_k z_{ik} \leq 1 \quad \forall i
  $$

#### 5. Money Aggregation
Money is calculated as:
$$
\text{money} = \sum_k c_k \cdot \text{character\_money}_k - \sum_{i,j} x_{ij} \cdot \text{Travel\_Cost}_{ij} - \sum_i \text{spend\_amount}_i
$$

#### 6. Spending Constraints
- Spending is allowed only if the player visits the island:
  $$
  s_i \leq y_i \quad \forall i
  $$
- Spending is capped by the total available money:
  $$
  \text{spend\_amount}_i \leq \text{money} \cdot s_i \quad \forall i
  $$
- Total spending cannot exceed available money:
  $$
  \sum_i \text{spend\_amount}_i \leq \text{money}
  $$

#### 7. Power Calculation


- Player's power on each island is calculated based on collected items, character modifiers, and money spent:

  $$
  p_i = \left( \sum_{k} z_{ik} \cdot \text{Item\_Power}_{ik} \cdot \left( \sum_{c} c_c \cdot \text{Modifier}_{c, k} \right) \right) + a \cdot \text{spend\_amount}_i
  $$

  Where:

  - $\text{Item\_Power}_{ik}$ : Base power of item $k$ on island $i$.
  - $\text{Modifier}_{c, k}$ : Modifier for character $c$ on item $k$.
  - $a$ : Exchange rate for money and power.

- Player can only get over the final island with at least 2000 units of power
$$
p_{14} \geq 8000
$$

#### 8. Adjusted Difficulties
Adjusted difficulties are defined as:
$$
\text{adjusted\_difficulties}_i \cdot p_i = \text{Island\_difficulty}_i
$$

---
### Calculation for Willingness

$$
\text{Willingness} = \frac{k \cdot (\text{Number of Islands Visited}) + l \cdot (\text{Number of Weapons Collected})}{\text{Total Gameplay Duration}}
$$


$$
= \space \frac{k \cdot \sum_{i=1}^{n} y_{i} + l \cdot \sum_{i=1}^{n} b_{i}}{\sum_{i=1}^{n} y_{i} \cdot \text{adjusted\_difficulties}_{i}}
$$

Coefficient for Number of Islands collected:  $k = 20$

- Different environments adhere to similar game settings, they typically don't have a strong impact on the overall gameplay experience.


Coefficient for Number of Weapons visited:  $l = 50$

- Different weapons can significantly alter the overall gameplay experience.


In [27]:
from gurobipy import Model, GRB, quicksum

### Basic Model: With No Talent and No Character Specified.

In [28]:
Island_name = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]

Ferry_ticket_price = [
    [0, 60, 60, 150, 174, 210, 246, 300, 360, 432, 480, 528, 570, 600, 900],
    [60, 0, 60, 120, 282, 132, 96, 102, 90, 264, 240, 360, 240, 1200, 900],
    [60, 60, 0, 132, 186, 54, 66, 174, 120, 30, 264, 180, 150, 390, 600],
    [270, 36, 150, 0, 294, 150, 120, 282, 192, 186, 180, 138, 258, 270, 360],
    [54, 72, 246, 174, 0, 84, 318, 132, 72, 78, 420, 420, 300, 396, 468],
    [126, 144, 192, 294, 210, 0, 180, 138, 222, 48, 138, 258, 90, 330, 390],
    [72, 114, 30, 168, 240, 246, 0, 210, 162, 306, 264, 270, 204, 390, 264],
    [204, 102, 60, 126, 54, 198, 300, 0, 180, 270, 324, 270, 390, 462, 480],
    [72, 114, 30, 168, 282, 234, 90, 360, 0, 306, 306, 300, 270, 330, 348],
    [72, 114, 30, 168, 282, 234, 90, 210, 432, 0, 132, 192, 132, 90, 330],
    [72, 114, 30, 168, 282, 234, 90, 210, 54, 480, 0, 132, 180, 60, 30],
    [72, 114, 30, 168, 282, 234, 90, 210, 54, 306, 528, 0, 150, 150, 180],
    [72, 114, 30, 168, 282, 234, 90, 210, 54, 306, 330, 570, 0, 60, 60],
    [72, 114, 30, 168, 282, 234, 90, 210, 54, 306, 396, 420, 60, 0, 60],
    [72, 114, 30, 168, 282, 234, 90, 210, 54, 306, 396, 468, 528, 900, 0]
]

Island_difficulty = [100, 300, 300, 500, 700, 700, 1000, 900, 1500, 2000, 2500, 4500, 4500, 5000, 10000]

Characters = ['thief', 'warrior', 'mage', 'archer', 'maid']

# Starting power for each character
start_power = [5, 10, 7, 8, 6.5]

# Starting money for each character
character_money = [4000, 2000, 1750, 1500, 3000]

Weapons = ['2hsword', 'bow', 'daggers', 'staff', 'mace']

# Weapons' modifier to each charafcter
Modifier = [
    [3, 1, 2, -1, 1],
    [2, 2, 2, 0, -1],
    [1, -1, 2, 3, 1],
    [3, 2.5, 1, -1, 1],
    [-1, -1, 1.5, 2, 2]
]# sword, bow, daggers, staff, mace for each class. Each row is a class and each column is a weapon type following the order listed

weapon_types = [
    ['2hsword', 'bow', 'daggers', 'staff', 'mace'],        # Island 1
    ['mace', '2hsword'],                # Island 2
    ['2hsword'],                        # Island 3
    ['bow', 'daggers', 'staff'],        # Island 4
    ['mace', '2hsword'],                # Island 5
    ['bow', 'staff', 'daggers', 'mace'],  # Island 6
    ['2hsword', 'bow'],                 # Island 7
    ['daggers'],                        # Island 8
    ['staff', '2hsword'],               # Island 9
    ['mace'],                           # Island 10
    ['bow', 'daggers', 'staff'],        # Island 11
    ['2hsword'],                        # Island 12
    ['mace', 'daggers', 'bow'],         # Island 13
    ['staff'],                          # Island 14
    ['2hsword', 'bow', 'daggers'],      # Island 15
]

base_weapon_power = [
    [3000, 3000, 3000, 3000, 3000],  # Weapons on island 1
    [2500, 2500],      # Weapons on island 2
    [2500],          # Weapons on island 3
    [2800, 2200, 2600],  # Weapons on island 4
    [3500, 2500],      # Weapons on island 5
    [3000, 2000, 3000, 4500],  # Weapons on island 6
    [2500, 2600],      # Weapons on island 7
    [6000],          # Weapons on island 8
    [7000, 6800],      # Weapons on island 9
    [6000],          # Weapons on island 10
    [7000, 6500, 6500],  # Weapons on island 11
    [7400],          # Weapons on island 12
    [6500, 7000, 6000],  # Weapons on island 13
    [8000],          # Weapons on island 14
    [5000, 5000, 5000],  # Weapons on island 15
]

n = len(Island_name)

a = 1 # Exchange rate for money and power

# Coefficient for features when calculating gamers' Willingness to play 
k = 20 # Coefficient for Number of Islands collected: Different environments adhere to similar game settings, they typically don't have a strong impact on the overall gameplay experience.
l = 50 # Coefficient for Number of Weapons visited: Different weapons can significantly alter the overall gameplay experience.

In [ ]:
# Create model
m = Model("IslandOptimization")

# Decision variables
x = m.addVars(n, n, vtype=GRB.BINARY, name="x")  # Travel from i to j
y = m.addVars(n, vtype=GRB.BINARY, name="y")  # Whether to visit island i
z = m.addVars(n, len(base_weapon_power), vtype=GRB.BINARY, name="z")  # Item taken on island i
b = m.addVars(n, len(base_weapon_power), vtype=GRB.BINARY, name="b")  # Item in backpack
c = m.addVars(len(character_money), vtype=GRB.BINARY, name="c")  # Character selection
s = m.addVars(n, vtype=GRB.BINARY, name="s")  # Whether money is spent on island i
spend_amount = m.addVars(n, vtype=GRB.CONTINUOUS, name="spend_amount")  # Amount spent on island i
p = m.addVars(n, vtype=GRB.CONTINUOUS, name="p")  # Power on each island
adjusted_difficulties = m.addVars(n, vtype=GRB.CONTINUOUS, name="adjusted_difficulties")  # Adjusted difficulties
money = m.addVar(vtype=GRB.CONTINUOUS, name="money")  # Total money
u = m.addVars(n, vtype=GRB.CONTINUOUS, lb=0, ub=n, name="u") # the order of island i be visited

# ==================================================================================================================================
# Objective function: Minimize adjusted time on islands
m.setObjective(quicksum(y[i] * adjusted_difficulties[i] for i in range(n)), GRB.MINIMIZE)

# ==================================================================================================================================
# Constraints

# No Sub Tour----------------------------------------------------------------------------------------------------------------
# Fix the position of the start island
m.addConstr(u[0] == 0, name="start_position")

# Miller-Tucker-Zemlin (MTZ) subtour elimination constraints
for i in range(1, n-1):  # Exclude start and end node if needed
    for j in range(1, n-1):
        if i != j:
            m.addConstr(u[i] - u[j] + n*x[i,j] <= n - 1, name=f"MTZ_{i}_{j}")


# Character selection: Ensure exactly one character is selected.-----------------------------------------------------------
m.addConstr(c.sum() == 1, name="single_character")
# m.addConstr(c[] == 1, name="single_character") # Choose specific character for scenario analysis

# Flow constraints: Maintain continuity in the travel path.-------------------------------------------------------------------
# The number of departures from an island equals the number of arrivals.
m.addConstrs(quicksum(x[i, j] for j in range(n) if j != i) == quicksum(x[j, i] for j in range(n) if j != i) for i in range(n))

# Starting island: Ensure the journey starts from a predefined start island (island 0).
m.addConstr(quicksum(x[0, j] for j in range(1, n)) == 1, name="start_island")

# Ending island: Ensure the journey ends at a predefined end island (last island, n-1).
m.addConstr(quicksum(x[i, n - 1] for i in range(n - 1)) == 1, name="end_island")

m.addConstr(quicksum(x[i, 0] for i in range(n) if i != 14) == 0, name="go_back_to_starting_point")
m.addConstr(x[14, 0] == 1, name="final_to_start")

# Connect x and y to ensure travel corresponds to visited islands
m.addConstrs((quicksum(x[i, j] for j in range(n) if i != j) >= y[i] for i in range(n)), name="visit_outgoing")
m.addConstrs((quicksum(x[j, i] for j in range(n) if i != j) >= y[i] for i in range(n)), name="visit_incoming")

# Add constraint to prevent backward travel
m.addConstrs((x[i, j] + x[j, i] <= 1 for i in range(n) for j in range(n) if i != j), name="no_backward_travel")
# Ensure each island is visited at most once
m.addConstrs((quicksum(x[j, i] for j in range(n) if j != i) == y[i] for i in range(n)), name="single_incoming")
m.addConstrs((quicksum(x[i, j] for j in range(n) if j != i) == y[i] for i in range(n)), name="single_outgoing")

# Island visitation constraints-----------------------------------------------------------------------------------------
# Ensure that an item can only be collected if the corresponding island is visited.
m.addConstrs((z[i, k] <= y[i] for i in range(n) for k in range(len(base_weapon_power[i]))), name="item_visitation")

# Link the collected items to those in the backpack, ensuring consistency between collection and possession.
m.addConstrs((b[i, k] == z[i, k] for i in range(n) for k in range(len(base_weapon_power[i]))), name="backpack_item")

# Single item per island: Limit the player to one item per island.
m.addConstrs((quicksum(z[i, k] for k in range(len(base_weapon_power[i]))) <= 1 for i in range(n)), name="single_item_per_island")

# Money aggregation-----------------------------------------------------------------------------------------------------
# Track the player's total money:
# Starts with the money associated with the selected character, subtracts travel costs and spending.
travel_cost = quicksum(x[i, j] * Ferry_ticket_price[i][j] for i in range(n) for j in range(n) if i != j)
m.addConstr(
    money == quicksum(c[k] * character_money[k] for k in range(len(character_money)))
    - travel_cost
    - spend_amount.sum(),
    name="money_aggregation",
)

# Spending constraints--------------------------------------------------------------------------------------------------
# Ensure money can only be spent on an island if the player visits it.
m.addConstrs((s[i] <= y[i] for i in range(n)), name="spend_link_to_visit")

# Limit the amount spent on an island to the available money and allow spending only if s[i] = 1.
m.addConstrs((spend_amount[i] <= money * s[i] for i in range(n)), name="spend_limit")

# Ensure the total spending across all islands does not exceed the player's available money.
m.addConstr(spend_amount.sum() <= money, name="total_spending_limit")

# Power calculation per island-------------------------------------------------------------------------------------------
# Power calculation per island with character-specific modifiers
m.addConstrs(
    (
        p[i] == quicksum(
            z[i, k] * base_weapon_power[i][k] * quicksum(
                c[char] * Modifier[char][weapon_types[i].index(Weapons[k])] 
                for char in range(len(Characters)) if Weapons[k] in weapon_types[i]
            )
            for k in range(len(base_weapon_power[i]))  # Explicitly loop over items (k) on island (i)
        ) + a * spend_amount[i]
    )
    for i in range(n)
)

# Ensure power at the final island (Island n-1) is at least 8000
m.addConstr(p[n - 1] >= 8000, name="final_island_power")

# Adjusted difficulties-------------------------------------------------------------------------------------------------
# Calculate the adjusted difficulty for each island, scaled inversely by the player's power.
m.addConstrs((adjusted_difficulties[i] * p[i] == Island_difficulty[i] for i in range(n)), name="adjusted_difficulty")


# ================================================================================================================================
# Optimize the model and print result
m.optimize()

# Extract and display the solution if optimal
if m.status == GRB.OPTIMAL:
    # Objective value (shortest time in this case)
    shortest_time = m.objVal
    
    # Extract the optimal travel path
    optimal_paths = [(i, j) for i in range(n) for j in range(n) if x[i, j].x > 0.5]
    
    # Extract visited islands
    visited_islands = [i for i in range(n) if y[i].x > 0.5]
    
    # Extract details about spending on each island
    spent_islands = {
        i: {
            "spent": s[i].x > 0.5, 
            "amount": spend_amount[i].x
        } for i in range(n) if y[i].x > 0.5
    }
    
    # Extract the selected character
    selected_character = [k for k in range(len(character_money)) if c[k].x > 0.5][0]
    
    # Extract details about items collected on each island
    collected_items = {
        i: {
            "item_collected": k, 
            "value": base_weapon_power[i][k],
            "modifier": Modifier[selected_character][weapon_types[i].index(Weapons[k])] if Weapons[k] in weapon_types[i] else 0
        } for i in range(n)
        for k in range(len(base_weapon_power[i]))
        if z[i, k].x > 0.5
    }
    
    # Print results
    print("Optimal Route (Shortest Time):")
    print(f"  Shortest Time: {shortest_time:.2f}")
    for i, j in optimal_paths:
        print(f"  Travel from Island {i} to Island {j}")
    
    print("\nIsland Spending Details:")
    for i, details in spent_islands.items():
        if details["amount"] > 0:
            spent = "Yes"
        else:
            spent = "No"
        print(f"  Island {i}: Spent Money={spent}, Amount Spent={details['amount']:.2f}")

    print("\nCollected Items:")
    for i in range(n):
        if i in collected_items:
            item_details = collected_items[i]
            power_gain = item_details["value"] * item_details["modifier"]
            print(f"  Island {i}: Collected Item {item_details['item_collected']} (Value={item_details['value']}, Power Gain={power_gain:.2f})")
        else:
            print(f"  Island {i}: No Item Collected")
    
    print("\nSelected Character:")
    print(f"  Character {selected_character}")
    # Retrieve the variable values from the solution
    y_values = [y[i].x for i in range(n)]
    b_values = []
    for i in range(n):
        for kk in range(len(base_weapon_power[i])):
            b_values.append(b[i, kk].x)
    
    adjusted_difficulties_values = [adjusted_difficulties[i].x for i in range(n)]

    # Compute the willingness value using the solution
    willingness_value = (k * sum(y_values) + l * sum(b_values) )/ (sum(y_values[i] * adjusted_difficulties_values[i] for i in range(n)))

    print("Willingness Value:", willingness_value)
else:
    print("No optimal solution found.")


Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11.0 (22631.2))

CPU model: AMD Ryzen 5 5600U with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 551 rows, 771 columns and 2711 nonzeros
Model fingerprint: 0xf9ec234c
Model has 15 quadratic objective terms
Model has 42 quadratic constraints
Variable types: 61 continuous, 710 integer (710 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+03]
  QMatrix range    [1e+00, 2e+04]
  QLMatrix range   [1e+00, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e+00, 2e+00]
  Bounds range     [1e+00, 2e+01]
  RHS range        [1e+00, 8e+03]
  QRHS range       [1e+02, 1e+04]
Presolve removed 248 rows and 476 columns
Presolve time: 0.01s
Presolved: 823 rows, 519 columns, 2994 nonzeros
Presolved model has 66 SOS constraint(s)
Presolved model has 15 bilinear constraint(s)
         in product terms.
         Pre

### Talent 1: $1000 Extra Starting Money and Character Thief Specified.

In [ ]:
character_money_1 = [value + 1000 for value in character_money]
# Create model
m = Model("IslandOptimization")

# Decision variables
x = m.addVars(n, n, vtype=GRB.BINARY, name="x")  # Travel from i to j
y = m.addVars(n, vtype=GRB.BINARY, name="y")  # Whether to visit island i
z = m.addVars(n, len(base_weapon_power), vtype=GRB.BINARY, name="z")  # Item taken on island i
b = m.addVars(n, len(base_weapon_power), vtype=GRB.BINARY, name="b")  # Item in backpack
c = m.addVars(len(character_money_1), vtype=GRB.BINARY, name="c")  # Character selection
s = m.addVars(n, vtype=GRB.BINARY, name="s")  # Whether money is spent on island i
spend_amount = m.addVars(n, vtype=GRB.CONTINUOUS, name="spend_amount")  # Amount spent on island i
p = m.addVars(n, vtype=GRB.CONTINUOUS, name="p")  # Power on each island
adjusted_difficulties = m.addVars(n, vtype=GRB.CONTINUOUS, name="adjusted_difficulties")  # Adjusted difficulties
money = m.addVar(vtype=GRB.CONTINUOUS, name="money")  # Total money
u = m.addVars(n, vtype=GRB.CONTINUOUS, lb=0, ub=n, name="u") # the order of island i be visited

# ==================================================================================================================================
# Objective function: Minimize adjusted time on islands
m.setObjective(quicksum(y[i] * adjusted_difficulties[i] for i in range(n)), GRB.MINIMIZE)

# ==================================================================================================================================
# Constraints

# No Sub Tour----------------------------------------------------------------------------------------------------------------
# Fix the position of the start island
m.addConstr(u[0] == 0, name="start_position")

# Miller-Tucker-Zemlin (MTZ) subtour elimination constraints
for i in range(1, n-1):  # Exclude start and end node if needed
    for j in range(1, n-1):
        if i != j:
            m.addConstr(u[i] - u[j] + n*x[i,j] <= n - 1, name=f"MTZ_{i}_{j}")


# Character selection: Ensure exactly one character is selected.-----------------------------------------------------------
m.addConstr(c.sum() == 1, name="single_character")
m.addConstr(c[0] == 1, name="single_character") # Choose specific character for scenario analysis

# Flow constraints: Maintain continuity in the travel path.-------------------------------------------------------------------
# The number of departures from an island equals the number of arrivals.
m.addConstrs(quicksum(x[i, j] for j in range(n) if j != i) == quicksum(x[j, i] for j in range(n) if j != i) for i in range(n))

# Starting island: Ensure the journey starts from a predefined start island (island 0).
m.addConstr(quicksum(x[0, j] for j in range(1, n)) == 1, name="start_island")

# Ending island: Ensure the journey ends at a predefined end island (last island, n-1).
m.addConstr(quicksum(x[i, n - 1] for i in range(n - 1)) == 1, name="end_island")

m.addConstr(quicksum(x[i, 0] for i in range(n) if i != 14) == 0, name="go_back_to_starting_point")
m.addConstr(x[14, 0] == 1, name="final_to_start")

# Connect x and y to ensure travel corresponds to visited islands
m.addConstrs((quicksum(x[i, j] for j in range(n) if i != j) >= y[i] for i in range(n)), name="visit_outgoing")
m.addConstrs((quicksum(x[j, i] for j in range(n) if i != j) >= y[i] for i in range(n)), name="visit_incoming")

# Add constraint to prevent backward travel
m.addConstrs((x[i, j] + x[j, i] <= 1 for i in range(n) for j in range(n) if i != j), name="no_backward_travel")
# Ensure each island is visited at most once
m.addConstrs((quicksum(x[j, i] for j in range(n) if j != i) == y[i] for i in range(n)), name="single_incoming")
m.addConstrs((quicksum(x[i, j] for j in range(n) if j != i) == y[i] for i in range(n)), name="single_outgoing")

# Island visitation constraints-----------------------------------------------------------------------------------------
# Ensure that an item can only be collected if the corresponding island is visited.
m.addConstrs((z[i, k] <= y[i] for i in range(n) for k in range(len(base_weapon_power[i]))), name="item_visitation")

# Link the collected items to those in the backpack, ensuring consistency between collection and possession.
m.addConstrs((b[i, k] == z[i, k] for i in range(n) for k in range(len(base_weapon_power[i]))), name="backpack_item")

# Single item per island: Limit the player to one item per island.
m.addConstrs((quicksum(z[i, k] for k in range(len(base_weapon_power[i]))) <= 1 for i in range(n)), name="single_item_per_island")

# Money aggregation-----------------------------------------------------------------------------------------------------
# Track the player's total money:
# Starts with the money associated with the selected character, subtracts travel costs and spending.
travel_cost = quicksum(x[i, j] * Ferry_ticket_price[i][j] for i in range(n) for j in range(n) if i != j)
m.addConstr(
    money == quicksum(c[k] * character_money_1[k] for k in range(len(character_money_1)))
    - travel_cost
    - spend_amount.sum(),
    name="money_aggregation",
)

# Spending constraints--------------------------------------------------------------------------------------------------
# Ensure money can only be spent on an island if the player visits it.
m.addConstrs((s[i] <= y[i] for i in range(n)), name="spend_link_to_visit")

# Limit the amount spent on an island to the available money and allow spending only if s[i] = 1.
m.addConstrs((spend_amount[i] <= money * s[i] for i in range(n)), name="spend_limit")

# Ensure the total spending across all islands does not exceed the player's available money.
m.addConstr(spend_amount.sum() <= money, name="total_spending_limit")

# Power calculation per island-------------------------------------------------------------------------------------------
# Power calculation per island with character-specific modifiers
m.addConstrs(
    (
        p[i] == quicksum(
            z[i, k] * base_weapon_power[i][k] * quicksum(
                c[char] * Modifier[char][weapon_types[i].index(Weapons[k])] 
                for char in range(len(Characters)) if Weapons[k] in weapon_types[i]
            )
            for k in range(len(base_weapon_power[i]))  # Explicitly loop over items (k) on island (i)
        ) + a * spend_amount[i]
    )
    for i in range(n)
)

# Ensure power at the final island (Island n-1) is at least 8000
m.addConstr(p[n - 1] >= 8000, name="final_island_power")

# Adjusted difficulties-------------------------------------------------------------------------------------------------
# Calculate the adjusted difficulty for each island, scaled inversely by the player's power.
m.addConstrs((adjusted_difficulties[i] * p[i] == Island_difficulty[i] for i in range(n)), name="adjusted_difficulty")


# ================================================================================================================================
# Optimize the model and print result
m.optimize()

# Extract and display the solution if optimal
if m.status == GRB.OPTIMAL:
    # Objective value (shortest time in this case)
    shortest_time = m.objVal
    
    # Extract the optimal travel path
    optimal_paths = [(i, j) for i in range(n) for j in range(n) if x[i, j].x > 0.5]
    
    # Extract visited islands
    visited_islands = [i for i in range(n) if y[i].x > 0.5]
    
    # Extract details about spending on each island
    spent_islands = {
        i: {
            "spent": s[i].x > 0.5, 
            "amount": spend_amount[i].x
        } for i in range(n) if y[i].x > 0.5
    }
    
    # Extract the selected character
    selected_character = [k for k in range(len(character_money_1)) if c[k].x > 0.5][0]
    
    # Extract details about items collected on each island
    collected_items = {
        i: {
            "item_collected": k, 
            "value": base_weapon_power[i][k],
            "modifier": Modifier[selected_character][weapon_types[i].index(Weapons[k])] if Weapons[k] in weapon_types[i] else 0
        } for i in range(n)
        for k in range(len(base_weapon_power[i]))
        if z[i, k].x > 0.5
    }
    
    # Print results
    print("Optimal Route (Shortest Time):")
    print(f"  Shortest Time: {shortest_time:.2f}")
    for i, j in optimal_paths:
        print(f"  Travel from Island {i} to Island {j}")
    
    print("\nIsland Spending Details:")
    for i, details in spent_islands.items():
        if details["amount"] > 0:
            spent = "Yes"
        else:
            spent = "No"
        print(f"  Island {i}: Spent Money={spent}, Amount Spent={details['amount']:.2f}")

    print("\nCollected Items:")
    for i in range(n):
        if i in collected_items:
            item_details = collected_items[i]
            power_gain = item_details["value"] * item_details["modifier"]
            print(f"  Island {i}: Collected Item {item_details['item_collected']} (Value={item_details['value']}, Power Gain={power_gain:.2f})")
        else:
            print(f"  Island {i}: No Item Collected")
    
    print("\nSelected Character:")
    print(f"  Character {selected_character}")
    # Retrieve the variable values from the solution
    y_values = [y[i].x for i in range(n)]
    b_values = []
    for i in range(n):
        for kk in range(len(base_weapon_power[i])):
            b_values.append(b[i, kk].x)
    
    adjusted_difficulties_values = [adjusted_difficulties[i].x for i in range(n)]

    # Compute the willingness value using the solution
    willingness_value = (k * sum(y_values) + l * sum(b_values) )/ (sum(y_values[i] * adjusted_difficulties_values[i] for i in range(n)))

    print("Willingness Value:", willingness_value)
else:
    print("No optimal solution found.")


Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11.0 (22631.2))

CPU model: AMD Ryzen 5 5600U with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 552 rows, 771 columns and 2712 nonzeros
Model fingerprint: 0xe2d581b3
Model has 15 quadratic objective terms
Model has 42 quadratic constraints
Variable types: 61 continuous, 710 integer (710 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+03]
  QMatrix range    [1e+00, 2e+04]
  QLMatrix range   [1e+00, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e+00, 2e+00]
  Bounds range     [1e+00, 2e+01]
  RHS range        [1e+00, 8e+03]
  QRHS range       [1e+02, 1e+04]
Presolve removed 239 rows and 486 columns
Presolve time: 0.01s
Presolved: 444 rows, 370 columns, 1975 nonzeros
Presolved model has 56 SOS constraint(s)
Presolved model has 15 bilinear constraint(s)
         in product terms.
         Pre

### Talent 2: Item Value get 20% increased and Character Thief Specified.

In [ ]:
base_weapon_power_2 = [[value * 1.2 for value in sublist] for sublist in base_weapon_power]

# Create model
m = Model("IslandOptimization")

# Decision variables
x = m.addVars(n, n, vtype=GRB.BINARY, name="x")  # Travel from i to j
y = m.addVars(n, vtype=GRB.BINARY, name="y")  # Whether to visit island i
z = m.addVars(n, len(base_weapon_power_2), vtype=GRB.BINARY, name="z")  # Item taken on island i
b = m.addVars(n, len(base_weapon_power_2), vtype=GRB.BINARY, name="b")  # Item in backpack
c = m.addVars(len(character_money), vtype=GRB.BINARY, name="c")  # Character selection
s = m.addVars(n, vtype=GRB.BINARY, name="s")  # Whether money is spent on island i
spend_amount = m.addVars(n, vtype=GRB.CONTINUOUS, name="spend_amount")  # Amount spent on island i
p = m.addVars(n, vtype=GRB.CONTINUOUS, name="p")  # Power on each island
adjusted_difficulties = m.addVars(n, vtype=GRB.CONTINUOUS, name="adjusted_difficulties")  # Adjusted difficulties
money = m.addVar(vtype=GRB.CONTINUOUS, name="money")  # Total money
u = m.addVars(n, vtype=GRB.CONTINUOUS, lb=0, ub=n, name="u") # the order of island i be visited

# ==================================================================================================================================
# Objective function: Minimize adjusted time on islands
m.setObjective(quicksum(y[i] * adjusted_difficulties[i] for i in range(n)), GRB.MINIMIZE)

# ==================================================================================================================================
# Constraints

# No Sub Tour----------------------------------------------------------------------------------------------------------------
# Fix the position of the start island
m.addConstr(u[0] == 0, name="start_position")

# Miller-Tucker-Zemlin (MTZ) subtour elimination constraints
for i in range(1, n-1):  # Exclude start and end node if needed
    for j in range(1, n-1):
        if i != j:
            m.addConstr(u[i] - u[j] + n*x[i,j] <= n - 1, name=f"MTZ_{i}_{j}")


# Character selection: Ensure exactly one character is selected.-----------------------------------------------------------
m.addConstr(c.sum() == 1, name="single_character")
m.addConstr(c[0] == 1, name="single_character") # Choose specific character for scenario analysis

# Flow constraints: Maintain continuity in the travel path.-------------------------------------------------------------------
# The number of departures from an island equals the number of arrivals.
m.addConstrs(quicksum(x[i, j] for j in range(n) if j != i) == quicksum(x[j, i] for j in range(n) if j != i) for i in range(n))

# Starting island: Ensure the journey starts from a predefined start island (island 0).
m.addConstr(quicksum(x[0, j] for j in range(1, n)) == 1, name="start_island")

# Ending island: Ensure the journey ends at a predefined end island (last island, n-1).
m.addConstr(quicksum(x[i, n - 1] for i in range(n - 1)) == 1, name="end_island")

m.addConstr(quicksum(x[i, 0] for i in range(n) if i != 14) == 0, name="go_back_to_starting_point")
m.addConstr(x[14, 0] == 1, name="final_to_start")

# Connect x and y to ensure travel corresponds to visited islands
m.addConstrs((quicksum(x[i, j] for j in range(n) if i != j) >= y[i] for i in range(n)), name="visit_outgoing")
m.addConstrs((quicksum(x[j, i] for j in range(n) if i != j) >= y[i] for i in range(n)), name="visit_incoming")

# Add constraint to prevent backward travel
m.addConstrs((x[i, j] + x[j, i] <= 1 for i in range(n) for j in range(n) if i != j), name="no_backward_travel")
# Ensure each island is visited at most once
m.addConstrs((quicksum(x[j, i] for j in range(n) if j != i) == y[i] for i in range(n)), name="single_incoming")
m.addConstrs((quicksum(x[i, j] for j in range(n) if j != i) == y[i] for i in range(n)), name="single_outgoing")

# Island visitation constraints-----------------------------------------------------------------------------------------
# Ensure that an item can only be collected if the corresponding island is visited.
m.addConstrs((z[i, k] <= y[i] for i in range(n) for k in range(len(base_weapon_power_2[i]))), name="item_visitation")

# Link the collected items to those in the backpack, ensuring consistency between collection and possession.
m.addConstrs((b[i, k] == z[i, k] for i in range(n) for k in range(len(base_weapon_power_2[i]))), name="backpack_item")

# Single item per island: Limit the player to one item per island.
m.addConstrs((quicksum(z[i, k] for k in range(len(base_weapon_power_2[i]))) <= 1 for i in range(n)), name="single_item_per_island")

# Money aggregation-----------------------------------------------------------------------------------------------------
# Track the player's total money:
# Starts with the money associated with the selected character, subtracts travel costs and spending.
travel_cost = quicksum(x[i, j] * Ferry_ticket_price[i][j] for i in range(n) for j in range(n) if i != j)
m.addConstr(
    money == quicksum(c[k] * character_money[k] for k in range(len(character_money)))
    - travel_cost
    - spend_amount.sum(),
    name="money_aggregation",
)

# Spending constraints--------------------------------------------------------------------------------------------------
# Ensure money can only be spent on an island if the player visits it.
m.addConstrs((s[i] <= y[i] for i in range(n)), name="spend_link_to_visit")

# Limit the amount spent on an island to the available money and allow spending only if s[i] = 1.
m.addConstrs((spend_amount[i] <= money * s[i] for i in range(n)), name="spend_limit")

# Ensure the total spending across all islands does not exceed the player's available money.
m.addConstr(spend_amount.sum() <= money, name="total_spending_limit")

# Power calculation per island-------------------------------------------------------------------------------------------
# Power calculation per island with character-specific modifiers
m.addConstrs(
    (
        p[i] == quicksum(
            z[i, k] * base_weapon_power_2[i][k] * quicksum(
                c[char] * Modifier[char][weapon_types[i].index(Weapons[k])] 
                for char in range(len(Characters)) if Weapons[k] in weapon_types[i]
            )
            for k in range(len(base_weapon_power_2[i]))  # Explicitly loop over items (k) on island (i)
        ) + a * spend_amount[i]
    )
    for i in range(n)
)

# Ensure power at the final island (Island n-1) is at least 8000
m.addConstr(p[n - 1] >= 8000, name="final_island_power")

# Adjusted difficulties-------------------------------------------------------------------------------------------------
# Calculate the adjusted difficulty for each island, scaled inversely by the player's power.
m.addConstrs((adjusted_difficulties[i] * p[i] == Island_difficulty[i] for i in range(n)), name="adjusted_difficulty")


# ================================================================================================================================
# Optimize the model and print result
m.optimize()

# Extract and display the solution if optimal
if m.status == GRB.OPTIMAL:
    # Objective value (shortest time in this case)
    shortest_time = m.objVal
    
    # Extract the optimal travel path
    optimal_paths = [(i, j) for i in range(n) for j in range(n) if x[i, j].x > 0.5]
    
    # Extract visited islands
    visited_islands = [i for i in range(n) if y[i].x > 0.5]
    
    # Extract details about spending on each island
    spent_islands = {
        i: {
            "spent": s[i].x > 0.5, 
            "amount": spend_amount[i].x
        } for i in range(n) if y[i].x > 0.5
    }
    
    # Extract the selected character
    selected_character = [k for k in range(len(character_money)) if c[k].x > 0.5][0]
    
    # Extract details about items collected on each island
    collected_items = {
        i: {
            "item_collected": k, 
            "value": base_weapon_power_2[i][k],
            "modifier": Modifier[selected_character][weapon_types[i].index(Weapons[k])] if Weapons[k] in weapon_types[i] else 0
        } for i in range(n)
        for k in range(len(base_weapon_power_2[i]))
        if z[i, k].x > 0.5
    }
    
    # Print results
    print("Optimal Route (Shortest Time):")
    print(f"  Shortest Time: {shortest_time:.2f}")
    for i, j in optimal_paths:
        print(f"  Travel from Island {i} to Island {j}")
    
    print("\nIsland Spending Details:")
    for i, details in spent_islands.items():
        if details["amount"] > 0:
            spent = "Yes"
        else:
            spent = "No"
        print(f"  Island {i}: Spent Money={spent}, Amount Spent={details['amount']:.2f}")

    print("\nCollected Items:")
    for i in range(n):
        if i in collected_items:
            item_details = collected_items[i]
            power_gain = item_details["value"] * item_details["modifier"]
            print(f"  Island {i}: Collected Item {item_details['item_collected']} (Value={item_details['value']}, Power Gain={power_gain:.2f})")
        else:
            print(f"  Island {i}: No Item Collected")
    
    print("\nSelected Character:")
    print(f"  Character {selected_character}")
    # Retrieve the variable values from the solution
    y_values = [y[i].x for i in range(n)]
    b_values = []
    for i in range(n):
        for kk in range(len(base_weapon_power_2[i])):
            b_values.append(b[i, kk].x)
    
    adjusted_difficulties_values = [adjusted_difficulties[i].x for i in range(n)]

    # Compute the willingness value using the solution
    willingness_value = (k * sum(y_values) + l * sum(b_values) )/ (sum(y_values[i] * adjusted_difficulties_values[i] for i in range(n)))

    print("Willingness Value:", willingness_value)
else:
    print("No optimal solution found.")


Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11.0 (22631.2))

CPU model: AMD Ryzen 5 5600U with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 552 rows, 771 columns and 2712 nonzeros
Model fingerprint: 0x8867618a
Model has 15 quadratic objective terms
Model has 42 quadratic constraints
Variable types: 61 continuous, 710 integer (710 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+03]
  QMatrix range    [1e+00, 3e+04]
  QLMatrix range   [1e+00, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e+00, 2e+00]
  Bounds range     [1e+00, 2e+01]
  RHS range        [1e+00, 8e+03]
  QRHS range       [1e+02, 1e+04]
Presolve removed 239 rows and 486 columns
Presolve time: 0.01s
Presolved: 444 rows, 370 columns, 1975 nonzeros
Presolved model has 56 SOS constraint(s)
Presolved model has 15 bilinear constraint(s)
         in product terms.
         Pre

### Talent3: Travel cost get 40% off and Character Thief Specified.

In [ ]:
# Create model
m = Model("IslandOptimization")

# Decision variables
x = m.addVars(n, n, vtype=GRB.BINARY, name="x")  # Travel from i to j
y = m.addVars(n, vtype=GRB.BINARY, name="y")  # Whether to visit island i
z = m.addVars(n, len(base_weapon_power), vtype=GRB.BINARY, name="z")  # Item taken on island i
b = m.addVars(n, len(base_weapon_power), vtype=GRB.BINARY, name="b")  # Item in backpack
c = m.addVars(len(character_money), vtype=GRB.BINARY, name="c")  # Character selection
s = m.addVars(n, vtype=GRB.BINARY, name="s")  # Whether money is spent on island i
spend_amount = m.addVars(n, vtype=GRB.CONTINUOUS, name="spend_amount")  # Amount spent on island i
p = m.addVars(n, vtype=GRB.CONTINUOUS, name="p")  # Power on each island
adjusted_difficulties = m.addVars(n, vtype=GRB.CONTINUOUS, name="adjusted_difficulties")  # Adjusted difficulties
money = m.addVar(vtype=GRB.CONTINUOUS, name="money")  # Total money
u = m.addVars(n, vtype=GRB.CONTINUOUS, lb=0, ub=n, name="u") # the order of island i be visited

# ==================================================================================================================================
# Objective function: Minimize adjusted time on islands
m.setObjective(quicksum(y[i] * adjusted_difficulties[i] for i in range(n)), GRB.MINIMIZE)

# ==================================================================================================================================
# Constraints

# No Sub Tour----------------------------------------------------------------------------------------------------------------
# Fix the position of the start island
m.addConstr(u[0] == 0, name="start_position")

# Miller-Tucker-Zemlin (MTZ) subtour elimination constraints
for i in range(1, n-1):  # Exclude start and end node if needed
    for j in range(1, n-1):
        if i != j:
            m.addConstr(u[i] - u[j] + n*x[i,j] <= n - 1, name=f"MTZ_{i}_{j}")


# Character selection: Ensure exactly one character is selected.-----------------------------------------------------------
m.addConstr(c.sum() == 1, name="single_character")
m.addConstr(c[0] == 1, name="single_character") # Choose specific character for scenario analysis

# Flow constraints: Maintain continuity in the travel path.-------------------------------------------------------------------
# The number of departures from an island equals the number of arrivals.
m.addConstrs(quicksum(x[i, j] for j in range(n) if j != i) == quicksum(x[j, i] for j in range(n) if j != i) for i in range(n))

# Starting island: Ensure the journey starts from a predefined start island (island 0).
m.addConstr(quicksum(x[0, j] for j in range(1, n)) == 1, name="start_island")

# Ending island: Ensure the journey ends at a predefined end island (last island, n-1).
m.addConstr(quicksum(x[i, n - 1] for i in range(n - 1)) == 1, name="end_island")

m.addConstr(quicksum(x[i, 0] for i in range(n) if i != 14) == 0, name="go_back_to_starting_point")
m.addConstr(x[14, 0] == 1, name="final_to_start")

# Connect x and y to ensure travel corresponds to visited islands
m.addConstrs((quicksum(x[i, j] for j in range(n) if i != j) >= y[i] for i in range(n)), name="visit_outgoing")
m.addConstrs((quicksum(x[j, i] for j in range(n) if i != j) >= y[i] for i in range(n)), name="visit_incoming")

# Add constraint to prevent backward travel
m.addConstrs((x[i, j] + x[j, i] <= 1 for i in range(n) for j in range(n) if i != j), name="no_backward_travel")
# Ensure each island is visited at most once
m.addConstrs((quicksum(x[j, i] for j in range(n) if j != i) == y[i] for i in range(n)), name="single_incoming")
m.addConstrs((quicksum(x[i, j] for j in range(n) if j != i) == y[i] for i in range(n)), name="single_outgoing")

# Island visitation constraints-----------------------------------------------------------------------------------------
# Ensure that an item can only be collected if the corresponding island is visited.
m.addConstrs((z[i, k] <= y[i] for i in range(n) for k in range(len(base_weapon_power[i]))), name="item_visitation")

# Link the collected items to those in the backpack, ensuring consistency between collection and possession.
m.addConstrs((b[i, k] == z[i, k] for i in range(n) for k in range(len(base_weapon_power[i]))), name="backpack_item")

# Single item per island: Limit the player to one item per island.
m.addConstrs((quicksum(z[i, k] for k in range(len(base_weapon_power[i]))) <= 1 for i in range(n)), name="single_item_per_island")

# Money aggregation-----------------------------------------------------------------------------------------------------
# Track the player's total money:
# Starts with the money associated with the selected character, subtracts travel costs and spending.
travel_cost = quicksum(0.6 * x[i, j] * Ferry_ticket_price[i][j] for i in range(n) for j in range(n) if i != j)
m.addConstr(
    money == quicksum(c[k] * character_money[k] for k in range(len(character_money)))
    - travel_cost
    - spend_amount.sum(),
    name="money_aggregation",
)

# Spending constraints--------------------------------------------------------------------------------------------------
# Ensure money can only be spent on an island if the player visits it.
m.addConstrs((s[i] <= y[i] for i in range(n)), name="spend_link_to_visit")

# Limit the amount spent on an island to the available money and allow spending only if s[i] = 1.
m.addConstrs((spend_amount[i] <= money * s[i] for i in range(n)), name="spend_limit")

# Ensure the total spending across all islands does not exceed the player's available money.
m.addConstr(spend_amount.sum() <= money, name="total_spending_limit")

# Power calculation per island-------------------------------------------------------------------------------------------
# Power calculation per island with character-specific modifiers
m.addConstrs(
    (
        p[i] == quicksum(
            z[i, k] * base_weapon_power[i][k] * quicksum(
                c[char] * Modifier[char][weapon_types[i].index(Weapons[k])] 
                for char in range(len(Characters)) if Weapons[k] in weapon_types[i]
            )
            for k in range(len(base_weapon_power[i]))  # Explicitly loop over items (k) on island (i)
        ) + a * spend_amount[i]
    )
    for i in range(n)
)

# Ensure power at the final island (Island n-1) is at least 8000
m.addConstr(p[n - 1] >= 8000, name="final_island_power")

# Adjusted difficulties-------------------------------------------------------------------------------------------------
# Calculate the adjusted difficulty for each island, scaled inversely by the player's power.
m.addConstrs((adjusted_difficulties[i] * p[i] == Island_difficulty[i] for i in range(n)), name="adjusted_difficulty")


# ================================================================================================================================
# Optimize the model and print result
m.optimize()

# Extract and display the solution if optimal
if m.status == GRB.OPTIMAL:
    # Objective value (shortest time in this case)
    shortest_time = m.objVal
    
    # Extract the optimal travel path
    optimal_paths = [(i, j) for i in range(n) for j in range(n) if x[i, j].x > 0.5]
    
    # Extract visited islands
    visited_islands = [i for i in range(n) if y[i].x > 0.5]
    
    # Extract details about spending on each island
    spent_islands = {
        i: {
            "spent": s[i].x > 0.5, 
            "amount": spend_amount[i].x
        } for i in range(n) if y[i].x > 0.5
    }
    
    # Extract the selected character
    selected_character = [k for k in range(len(character_money)) if c[k].x > 0.5][0]
    
    # Extract details about items collected on each island
    collected_items = {
        i: {
            "item_collected": k, 
            "value": base_weapon_power[i][k],
            "modifier": Modifier[selected_character][weapon_types[i].index(Weapons[k])] if Weapons[k] in weapon_types[i] else 0
        } for i in range(n)
        for k in range(len(base_weapon_power[i]))
        if z[i, k].x > 0.5
    }
    
    # Print results
    print("Optimal Route (Shortest Time):")
    print(f"  Shortest Time: {shortest_time:.2f}")
    for i, j in optimal_paths:
        print(f"  Travel from Island {i} to Island {j}")
    
    print("\nIsland Spending Details:")
    for i, details in spent_islands.items():
        if details["amount"] > 0:
            spent = "Yes"
        else:
            spent = "No"
        print(f"  Island {i}: Spent Money={spent}, Amount Spent={details['amount']:.2f}")

    print("\nCollected Items:")
    for i in range(n):
        if i in collected_items:
            item_details = collected_items[i]
            power_gain = item_details["value"] * item_details["modifier"]
            print(f"  Island {i}: Collected Item {item_details['item_collected']} (Value={item_details['value']}, Power Gain={power_gain:.2f})")
        else:
            print(f"  Island {i}: No Item Collected")
    
    print("\nSelected Character:")
    print(f"  Character {selected_character}")
    # Retrieve the variable values from the solution
    y_values = [y[i].x for i in range(n)]
    b_values = []
    for i in range(n):
        for kk in range(len(base_weapon_power[i])):
            b_values.append(b[i, kk].x)
    
    adjusted_difficulties_values = [adjusted_difficulties[i].x for i in range(n)]

    # Compute the willingness value using the solution
    willingness_value = (k * sum(y_values) + l * sum(b_values) )/ (sum(y_values[i] * adjusted_difficulties_values[i] for i in range(n)))

    print("Willingness Value:", willingness_value)
else:
    print("No optimal solution found.")

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11.0 (22631.2))

CPU model: AMD Ryzen 5 5600U with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads



Optimize a model with 552 rows, 771 columns and 2712 nonzeros
Model fingerprint: 0x91707b10
Model has 15 quadratic objective terms
Model has 42 quadratic constraints
Variable types: 61 continuous, 710 integer (710 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+03]
  QMatrix range    [1e+00, 2e+04]
  QLMatrix range   [1e+00, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e+00, 2e+00]
  Bounds range     [1e+00, 2e+01]
  RHS range        [1e+00, 8e+03]
  QRHS range       [1e+02, 1e+04]
Presolve removed 239 rows and 486 columns
Presolve time: 0.01s
Presolved: 444 rows, 370 columns, 1975 nonzeros
Presolved model has 56 SOS constraint(s)
Presolved model has 15 bilinear constraint(s)
         in product terms.
         Presolve was not able to compute smaller bounds for these variables.
         Consider bounding these variables or reformulating the model.


Solving non-convex MIQCP

Variable types: 113 continuous, 257 integer (257 binary)

Root relaxation: objec

### Talent 4: Difficulty decreased by 8% and Character Thief Specified.

In [ ]:
Island_difficulty_4 = [value * 0.92 for value in Island_difficulty]
# Create model
m = Model("IslandOptimization")

# Decision variables
x = m.addVars(n, n, vtype=GRB.BINARY, name="x")  # Travel from i to j
y = m.addVars(n, vtype=GRB.BINARY, name="y")  # Whether to visit island i
z = m.addVars(n, len(base_weapon_power), vtype=GRB.BINARY, name="z")  # Item taken on island i
b = m.addVars(n, len(base_weapon_power), vtype=GRB.BINARY, name="b")  # Item in backpack
c = m.addVars(len(character_money), vtype=GRB.BINARY, name="c")  # Character selection
s = m.addVars(n, vtype=GRB.BINARY, name="s")  # Whether money is spent on island i
spend_amount = m.addVars(n, vtype=GRB.CONTINUOUS, name="spend_amount")  # Amount spent on island i
p = m.addVars(n, vtype=GRB.CONTINUOUS, name="p")  # Power on each island
adjusted_difficulties = m.addVars(n, vtype=GRB.CONTINUOUS, name="adjusted_difficulties")  # Adjusted difficulties
money = m.addVar(vtype=GRB.CONTINUOUS, name="money")  # Total money
u = m.addVars(n, vtype=GRB.CONTINUOUS, lb=0, ub=n, name="u") # the order of island i be visited

# ==================================================================================================================================
# Objective function: Minimize adjusted time on islands
m.setObjective(quicksum(y[i] * adjusted_difficulties[i] for i in range(n)), GRB.MINIMIZE)

# ==================================================================================================================================
# Constraints

# No Sub Tour----------------------------------------------------------------------------------------------------------------
# Fix the position of the start island
m.addConstr(u[0] == 0, name="start_position")

# Miller-Tucker-Zemlin (MTZ) subtour elimination constraints
for i in range(1, n-1):  # Exclude start and end node if needed
    for j in range(1, n-1):
        if i != j:
            m.addConstr(u[i] - u[j] + n*x[i,j] <= n - 1, name=f"MTZ_{i}_{j}")


# Character selection: Ensure exactly one character is selected.-----------------------------------------------------------
m.addConstr(c.sum() == 1, name="single_character")
m.addConstr(c[0] == 1, name="single_character") # Choose specific character for scenario analysis

# Flow constraints: Maintain continuity in the travel path.-------------------------------------------------------------------
# The number of departures from an island equals the number of arrivals.
m.addConstrs(quicksum(x[i, j] for j in range(n) if j != i) == quicksum(x[j, i] for j in range(n) if j != i) for i in range(n))

# Starting island: Ensure the journey starts from a predefined start island (island 0).
m.addConstr(quicksum(x[0, j] for j in range(1, n)) == 1, name="start_island")

# Ending island: Ensure the journey ends at a predefined end island (last island, n-1).
m.addConstr(quicksum(x[i, n - 1] for i in range(n - 1)) == 1, name="end_island")

m.addConstr(quicksum(x[i, 0] for i in range(n) if i != 14) == 0, name="go_back_to_starting_point")
m.addConstr(x[14, 0] == 1, name="final_to_start")

# Connect x and y to ensure travel corresponds to visited islands
m.addConstrs((quicksum(x[i, j] for j in range(n) if i != j) >= y[i] for i in range(n)), name="visit_outgoing")
m.addConstrs((quicksum(x[j, i] for j in range(n) if i != j) >= y[i] for i in range(n)), name="visit_incoming")

# Add constraint to prevent backward travel
m.addConstrs((x[i, j] + x[j, i] <= 1 for i in range(n) for j in range(n) if i != j), name="no_backward_travel")
# Ensure each island is visited at most once
m.addConstrs((quicksum(x[j, i] for j in range(n) if j != i) == y[i] for i in range(n)), name="single_incoming")
m.addConstrs((quicksum(x[i, j] for j in range(n) if j != i) == y[i] for i in range(n)), name="single_outgoing")

# Island visitation constraints-----------------------------------------------------------------------------------------
# Ensure that an item can only be collected if the corresponding island is visited.
m.addConstrs((z[i, k] <= y[i] for i in range(n) for k in range(len(base_weapon_power[i]))), name="item_visitation")

# Link the collected items to those in the backpack, ensuring consistency between collection and possession.
m.addConstrs((b[i, k] == z[i, k] for i in range(n) for k in range(len(base_weapon_power[i]))), name="backpack_item")

# Single item per island: Limit the player to one item per island.
m.addConstrs((quicksum(z[i, k] for k in range(len(base_weapon_power[i]))) <= 1 for i in range(n)), name="single_item_per_island")

# Money aggregation-----------------------------------------------------------------------------------------------------
# Track the player's total money:
# Starts with the money associated with the selected character, subtracts travel costs and spending.
travel_cost = quicksum(x[i, j] * Ferry_ticket_price[i][j] for i in range(n) for j in range(n) if i != j)
m.addConstr(
    money == quicksum(c[k] * character_money[k] for k in range(len(character_money)))
    - travel_cost
    - spend_amount.sum(),
    name="money_aggregation",
)

# Spending constraints--------------------------------------------------------------------------------------------------
# Ensure money can only be spent on an island if the player visits it.
m.addConstrs((s[i] <= y[i] for i in range(n)), name="spend_link_to_visit")

# Limit the amount spent on an island to the available money and allow spending only if s[i] = 1.
m.addConstrs((spend_amount[i] <= money * s[i] for i in range(n)), name="spend_limit")

# Ensure the total spending across all islands does not exceed the player's available money.
m.addConstr(spend_amount.sum() <= money, name="total_spending_limit")

# Power calculation per island-------------------------------------------------------------------------------------------
# Power calculation per island with character-specific modifiers
m.addConstrs(
    (
        p[i] == quicksum(
            z[i, k] * base_weapon_power[i][k] * quicksum(
                c[char] * Modifier[char][weapon_types[i].index(Weapons[k])] 
                for char in range(len(Characters)) if Weapons[k] in weapon_types[i]
            )
            for k in range(len(base_weapon_power[i]))  # Explicitly loop over items (k) on island (i)
        ) + a * spend_amount[i]
    )
    for i in range(n)
)

# Ensure power at the final island (Island n-1) is at least 8000
m.addConstr(p[n - 1] >= 8000, name="final_island_power")

# Adjusted difficulties-------------------------------------------------------------------------------------------------
# Calculate the adjusted difficulty for each island, scaled inversely by the player's power.
m.addConstrs((adjusted_difficulties[i] * p[i] == Island_difficulty_4[i] for i in range(n)), name="adjusted_difficulty")


# ================================================================================================================================
# Optimize the model and print result
m.optimize()

# Extract and display the solution if optimal
if m.status == GRB.OPTIMAL:
    # Objective value (shortest time in this case)
    shortest_time = m.objVal
    
    # Extract the optimal travel path
    optimal_paths = [(i, j) for i in range(n) for j in range(n) if x[i, j].x > 0.5]
    
    # Extract visited islands
    visited_islands = [i for i in range(n) if y[i].x > 0.5]
    
    # Extract details about spending on each island
    spent_islands = {
        i: {
            "spent": s[i].x > 0.5, 
            "amount": spend_amount[i].x
        } for i in range(n) if y[i].x > 0.5
    }
    
    # Extract the selected character
    selected_character = [k for k in range(len(character_money)) if c[k].x > 0.5][0]
    
    # Extract details about items collected on each island
    collected_items = {
        i: {
            "item_collected": k, 
            "value": base_weapon_power[i][k],
            "modifier": Modifier[selected_character][weapon_types[i].index(Weapons[k])] if Weapons[k] in weapon_types[i] else 0
        } for i in range(n)
        for k in range(len(base_weapon_power[i]))
        if z[i, k].x > 0.5
    }
    
    # Print results
    print("Optimal Route (Shortest Time):")
    print(f"  Shortest Time: {shortest_time:.2f}")
    for i, j in optimal_paths:
        print(f"  Travel from Island {i} to Island {j}")
    
    print("\nIsland Spending Details:")
    for i, details in spent_islands.items():
        if details["amount"] > 0:
            spent = "Yes"
        else:
            spent = "No"
        print(f"  Island {i}: Spent Money={spent}, Amount Spent={details['amount']:.2f}")

    print("\nCollected Items:")
    for i in range(n):
        if i in collected_items:
            item_details = collected_items[i]
            power_gain = item_details["value"] * item_details["modifier"]
            print(f"  Island {i}: Collected Item {item_details['item_collected']} (Value={item_details['value']}, Power Gain={power_gain:.2f})")
        else:
            print(f"  Island {i}: No Item Collected")
    
    print("\nSelected Character:")
    print(f"  Character {selected_character}")
    # Retrieve the variable values from the solution
    y_values = [y[i].x for i in range(n)]
    b_values = []
    for i in range(n):
        for kk in range(len(base_weapon_power[i])):
            b_values.append(b[i, kk].x)
    
    adjusted_difficulties_values = [adjusted_difficulties[i].x for i in range(n)]

    # Compute the willingness value using the solution
    willingness_value = (k * sum(y_values) + l * sum(b_values) )/ (sum(y_values[i] * adjusted_difficulties_values[i] for i in range(n)))

    print("Willingness Value:", willingness_value)
else:
    print("No optimal solution found.")


Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11.0 (22631.2))

CPU model: AMD Ryzen 5 5600U with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 552 rows, 771 columns and 2712 nonzeros
Model fingerprint: 0xe7d1adb3
Model has 15 quadratic objective terms
Model has 42 quadratic constraints
Variable types: 61 continuous, 710 integer (710 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+03]
  QMatrix range    [1e+00, 2e+04]
  QLMatrix range   [1e+00, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e+00, 2e+00]
  Bounds range     [1e+00, 2e+01]
  RHS range        [1e+00, 8e+03]
  QRHS range       [9e+01, 9e+03]
Presolve removed 239 rows and 486 columns
Presolve time: 0.01s
Presolved: 444 rows, 370 columns, 1975 nonzeros
Presolved model has 56 SOS constraint(s)
Presolved model has 15 bilinear constraint(s)
         in product terms.
         Pre